In [1]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

from processing import load_parquet_files

In [3]:
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / 'datos' / 'datos_prueba'
assert DATA_DIR.is_dir()

In [6]:
df = load_parquet_files(DATA_DIR)

In [8]:
df

,id,text,created_at,lang,entities.urls,entities.user_mentions,entities.hashtags,user.id,user.description,user.location,...,user.default_profile_image,is_retweet,is_quote,is_reply,in_reply_to_user_id,in_reply_to_status_id,quote.id,quote.user.id,rt.id,rt.user.id
0,1212536471674085376,@InaGroovie Ufff! Mejor no pensar en ello. Dis...,Thu Jan 02 00:49:57 +0000 2020,es,,37970881,,36792273,obrera de archivo y de historia.\nla historia ...,Chile,...,False,False,False,True,37970881,1212535490584498176,0,0,0,0
1,1212536472307470336,@jorgeerrazuriz @Carlossassarini Jaja jajajaja...,Thu Jan 02 00:49:57 +0000 2020,es,,197455385|161081949,,1193609731300089856,triatleta,,...,False,False,False,True,197455385,1212447143321055233,0,0,0,0
2,1212536472429043713,"@CHVNoticias Viejo culiao, chupapico que se me...",Thu Jan 02 00:49:57 +0000 2020,es,,51326671,,1136071561776160769,,,...,False,False,False,True,51326671,1212522225674457090,0,0,0,0
3,1212439892225142784,#ServicioSocial 🔴 Familiares de Brenda Nayeli ...,Wed Jan 01 18:26:11 +0000 2020,es,,,ServicioSocial,236602067,Somos el periódico de mayor presencia en Puebl...,"Puebla, Puebla, México",...,False,False,False,False,0,0,0,0,0,0
4,1212536472571482112,#ServicioSocial 🔴 Familiares de Brenda Nayeli ...,Thu Jan 02 00:49:57 +0000 2020,es,,,ServicioSocial,3153212562,,,...,False,True,False,False,0,0,0,0,1212439892225142784,236602067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884388,1212395540656340992,Que gentesita...\nPuros criminales!\n\nSi YO f...,Wed Jan 01 15:29:56 +0000 2020,es,,69467961,,2305949045,Venezuelan journalist and videographer (Eng/Es...,,...,False,False,True,False,0,0,1212203196438994945,69467961,0,0
884389,1212395540954124294,#Mosciatti en este punto de la historia de chi...,Wed Jan 01 15:29:56 +0000 2020,es,,,Mosciatti,117139349,Un hombre realmente muy sencillo. \n\n Aguante...,,...,False,True,False,False,0,0,0,0,1212320911845679105,1187197938172334080
884390,1212174283503525890,Su currículum se parece más al de un terrorist...,Wed Jan 01 00:50:44 +0000 2020,es,https://twitter.com/fontirroigpablo/status/121...,70707699,,122172956,Cuando en el mundo hay paz y tranquilidad; flo...,Chile,...,False,False,True,False,0,0,1212060045929791489,70707699,0,0
884391,1212395540970909699,Su currículum se parece más al de un terrorist...,Wed Jan 01 15:29:56 +0000 2020,es,https://twitter.com/fontirroigpablo/status/121...,70707699,,3161804152,,,...,False,True,True,False,0,0,1212060045929791489,70707699,1212174283503525890,122172956
